In [4]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
val spark = SparkSession.builder.master("local[*]").appName("dataProcessing").getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@5bfe7be


org.apache.spark.sql.SparkSession@5bfe7be

In [13]:
import java.io.File
import scala.io.Source

val path = System.getProperty("user.dir")
val seasons = 2002 to 2002
val teams = 1 to 30

for (i <- seasons){
    for (j <- teams){
        val file = path + "/season_averages/" + i + "/" + j + ".json"
        try{
            var df_stats = spark.read.json(file)
            var elem = df_stats.select(explode($"data").as("data"))
            var minutes = elem.select("data.min")
            val split_min = elem.withColumn("seconds", split($"data.min", ":")(0) * 60 + split($"data.min", ":")(1))
            val best_12 = split_min.orderBy(desc("seconds")).limit(12)
            val best_12_struct = best_12.withColumn("row", struct(col("data.ast"), col("data.blk"), col("data.dreb"),
                                                                  col("data.fg3_pct"), col("data.fg3a"), 
                                                                  col("data.fg3m"), col("data.fg_pct"), 
                                                                  col("data.fga"), col("data.fgm"), 
                                                                  col("data.ft_pct"), col("data.fta"),
                                                                  col("data.ftm"), col("data.games_played"), 
                                                                  col("seconds"), col("data.oreb"), col("data.pf"),
                                                                  col("data.player_id"), 
                                                                  col("data.pts"), col("data.reb"), 
                                                                  col("data.season"), col("data.stl"), 
                                                                  col("data.turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_data = best_12_struct.select("data")
            best_12_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j)
            
            //Modifying certain averages to be scaled to 36 minutes
            val best_12_36 = best_12.withColumn("ast", col("ast") / col("seconds") * 2160)
            .withColumn("blk", col("blk") / col("seconds") * 2160)
            .withColumn("dreb", col("dreb") / col("seconds") * 2160)
            .withColumn("fg3a", col("fg3a") / col("seconds") * 2160)
            .withColumn("fg3m", col("fg3m") / col("seconds") * 2160)
            .withColumn("fga", col("fga") / col("seconds") * 2160)
            .withColumn("fgm", col("fgm") / col("seconds") * 2160)
            .withColumn("fta", col("fta") / col("seconds") * 2160)
            .withColumn("ftm", col("ftm") / col("seconds") * 2160)
            .withColumn("oreb", col("oreb") / col("seconds") * 2160)
            .withColumn("pf", col("pf") / col("seconds") * 2160)
            .withColumn("pts", col("pts") / col("seconds") * 2160)
            .withColumn("reb", col("reb") / col("seconds") * 2160)
            .withColumn("seconds", col("seconds") / col("seconds") * 2160)
            .withColumn("stl", col("stl") / col("seconds") * 2160)
            .withColumn("turnover", col("turnover") / col("seconds") * 2160)
            
            val best_12_36_struct = best_12_36.withColumn("row", struct(col("ast"), col("blk"), col("dreb"), col("fg3_pct"), col("fg3a"), 
                                                                   col("fg3m"), col("fg_pct"), col("fga"), col("fgm"), 
                                                                   col("ft_pct"), col("fta"), col("ftm"), col("games_played"), 
                                                                   col("seconds"), col("oreb"), col("pf"), col("player_id"), 
                                                                   col("pts"), col("reb"), col("season"), col("stl"), col("turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_36_data = best_12_36_struct.select("data")
            best_12_36_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j + "-36")
            println("File " + file + " saved")
            
        }
        catch{
            case e: org.apache.spark.sql.AnalysisException => println("File " + file + " not found")
        }
    } 
}

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ast: double (nullable = true)
 |    |    |-- blk: double (nullable = true)
 |    |    |-- dreb: double (nullable = true)
 |    |    |-- fg3_pct: double (nullable = true)
 |    |    |-- fg3a: double (nullable = true)
 |    |    |-- fg3m: double (nullable = true)
 |    |    |-- fg_pct: double (nullable = true)
 |    |    |-- fga: double (nullable = true)
 |    |    |-- fgm: double (nullable = true)
 |    |    |-- ft_pct: double (nullable = true)
 |    |    |-- fta: double (nullable = true)
 |    |    |-- ftm: double (nullable = true)
 |    |    |-- games_played: long (nullable = true)
 |    |    |-- seconds: double (nullable = true)
 |    |    |-- oreb: double (nullable = true)
 |    |    |-- pf: double (nullable = true)
 |    |    |-- player_id: long (nullable = true)
 |    |    |-- pts: double (nullable = true)
 |    |    |-- reb: double (nullable = true)
 |    |    |-- season: long (

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ast: double (nullable = true)
 |    |    |-- blk: double (nullable = true)
 |    |    |-- dreb: double (nullable = true)
 |    |    |-- fg3_pct: double (nullable = true)
 |    |    |-- fg3a: double (nullable = true)
 |    |    |-- fg3m: double (nullable = true)
 |    |    |-- fg_pct: double (nullable = true)
 |    |    |-- fga: double (nullable = true)
 |    |    |-- fgm: double (nullable = true)
 |    |    |-- ft_pct: double (nullable = true)
 |    |    |-- fta: double (nullable = true)
 |    |    |-- ftm: double (nullable = true)
 |    |    |-- games_played: long (nullable = true)
 |    |    |-- seconds: double (nullable = true)
 |    |    |-- oreb: double (nullable = true)
 |    |    |-- pf: double (nullable = true)
 |    |    |-- player_id: long (nullable = true)
 |    |    |-- pts: double (nullable = true)
 |    |    |-- reb: double (nullable = true)
 |    |    |-- season: long (

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ast: double (nullable = true)
 |    |    |-- blk: double (nullable = true)
 |    |    |-- dreb: double (nullable = true)
 |    |    |-- fg3_pct: double (nullable = true)
 |    |    |-- fg3a: double (nullable = true)
 |    |    |-- fg3m: double (nullable = true)
 |    |    |-- fg_pct: double (nullable = true)
 |    |    |-- fga: double (nullable = true)
 |    |    |-- fgm: double (nullable = true)
 |    |    |-- ft_pct: double (nullable = true)
 |    |    |-- fta: double (nullable = true)
 |    |    |-- ftm: double (nullable = true)
 |    |    |-- games_played: long (nullable = true)
 |    |    |-- seconds: double (nullable = true)
 |    |    |-- oreb: double (nullable = true)
 |    |    |-- pf: double (nullable = true)
 |    |    |-- player_id: long (nullable = true)
 |    |    |-- pts: double (nullable = true)
 |    |    |-- reb: double (nullable = true)
 |    |    |-- season: long (

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ast: double (nullable = true)
 |    |    |-- blk: double (nullable = true)
 |    |    |-- dreb: double (nullable = true)
 |    |    |-- fg3_pct: double (nullable = true)
 |    |    |-- fg3a: double (nullable = true)
 |    |    |-- fg3m: double (nullable = true)
 |    |    |-- fg_pct: double (nullable = true)
 |    |    |-- fga: double (nullable = true)
 |    |    |-- fgm: double (nullable = true)
 |    |    |-- ft_pct: double (nullable = true)
 |    |    |-- fta: double (nullable = true)
 |    |    |-- ftm: double (nullable = true)
 |    |    |-- games_played: long (nullable = true)
 |    |    |-- seconds: double (nullable = true)
 |    |    |-- oreb: double (nullable = true)
 |    |    |-- pf: double (nullable = true)
 |    |    |-- player_id: long (nullable = true)
 |    |    |-- pts: double (nullable = true)
 |    |    |-- reb: double (nullable = true)
 |    |    |-- season: long (

java.lang.InterruptedException: None